# ZCU208 SFP Self-Test
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

This notebook performs a self-test of the ZCU208 2x2 SFP configuration in loopback mode.
Only the upper-right SFP is active in this overlay.

![SFP_2x2_cage](data/ZCU208_SFP_2x2_Cage_with_LoopbackAtUpperRight.jpg)

## Aims
* Describe the `xxv_ethernet` Python package that is required to control and
  configure the socket from Jupyter
* The xxv_ethernet configures one of the four 25G SFPs as 25G interface.
* Demonstrate loopback performance.

## Prerequisites
* Jumpers must be removed at J32, J35, J39, J44.  Placing them would disable the SFP socket.
* External loopback requires an SFP loopback module. This design was verified with the <A HREF='https://www.fs.com/products/109371.html'>FS SFP28-LB</A>.

## References
* [Xilinx, Inc, "ZCU208 Evaluation Board User Guide", UG1410, v1.0, July 2020](https://www.xilinx.com/support/documentation/boards_and_kits/zcu208/ug1410-zcu208-eval-bd.pdf)
* [Xilinx, Inc, "10G/25G High Speed Ethernet Subsystem v4.1", PG210, v4.1, May 2022](https://docs.xilinx.com/r/en-US/pg210-25g-ethernet)

## Revision History
* v1.0 | 27/09/2022 | Revised for the ZCU208.


In [1]:
import pynq
import numpy as np

from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')
dma = base.OffloadEngine.axi_dma_0
xxv = base.OffloadEngine.xxv_ethernet_0

In [ ]:
# Internal Loopback = 1
# External Loopback (requires SFP present at SFP00 upper-right) = 0
xxv.register_map.MODE_REG.ctl_local_loopback = 1 # change to 0 for external

In [3]:
# Reset recommended after changing to/from Loopback mode
# Verify SFP module properly inserted before attempting external loopback
xxv.register_map.GT_RESET_REG = 0x01
xxv.register_map.RESET_REG = 0x03
xxv.register_map.GT_RESET_REG = 0x00
xxv.register_map.RESET_REG = 0x00

In [4]:
# Some ASCII artwork
raw_str = ' /$$$$$$$  /$$     /$$ /$$   /$$  /$$$$$$ \n| $$__  $$|  $$   /$$/| $$$ | $$ /$$__  $$\n| $$  \\ $$ \\  $$ /$$/ | $$$$| $$| $$  \\ $$\n| $$$$$$$/  \\  $$$$/  | $$ $$ $$| $$  | $$\n| $$____/    \\  $$/   | $$  $$$$| $$  | $$\n| $$          | $$    | $$\\  $$$| $$/$$ $$\n| $$          | $$    | $$ \\  $$|  $$$$$$/\n|__/          |__/    |__/  \\__/ \\____ $$$\n                                      \\__/\n\n                                          \n                                         '
packets_in = np.array(list(raw_str.encode()))
# Allocate input and output buffers for transfer
dma_buf_in = pynq.allocate(packets_in.shape[0])
dma_buf_out = pynq.allocate(packets_in.shape[0])
# Load data into input buffer
dma_buf_in[:]=packets_in

In [5]:
# Now initiate the transmission of a packet by sending to DMA engine
# The receive packet is captured and stored back by the DMA engine.
dma.sendchannel.transfer(dma_buf_in)
dma.recvchannel.transfer(dma_buf_out)
dma.sendchannel.wait()
dma.recvchannel.wait()

In [6]:
# Decode output data
packets_out = bytes(dma_buf_out)
print(packets_out.decode())

    /   $   $   $   $   $   $   $           /   $   $                       /   $   $       /   $   $               /   $   $           /   $   $   $   $   $   $       
   |       $   $   _   _           $   $   |           $   $               /   $   $   /   |       $   $   $       |       $   $       /   $   $   _   _           $   $   
   |       $   $           \       $   $       \           $   $       /   $   $   /       |       $   $   $   $   |       $   $   |       $   $           \       $   $   
   |       $   $   $   $   $   $   $   /           \           $   $   $   $   /           |       $   $       $   $       $   $   |       $   $           |       $   $   
   |       $   $   _   _   _   _   /                   \           $   $   /               |       $   $           $   $   $   $   |       $   $           |       $   $   
   |       $   $                                           |       $   $                   |       $   $   \           $   $   $   |       $   

In [7]:
# Another way to verify core functionality
np.array_equal(dma_buf_in,dma_buf_out)
# returns True if the xmit and recv buffers are equal

True

# Conclusion:
* Verify that the upper-right SFP socket functions for both internal and external loopback mode configurations.